# ***Movie Recommendation System using weighted hybrid technique***

## Importing libraries

In [ ]:
import pandas as pd
import numpy as np

***Data Source:*** https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv

In [ ]:
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies_df = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

How does the credits data look like ?

In [ ]:
credits.head()

Cast and Crew columns are in JSON format

How does the movies data look ?

In [ ]:
movies_df.head()

Dimensions of the data : 

In [ ]:
print("Credits : ", credits.shape)
print("Movies : ", movies_df.shape)

## Merging both datasets based on "id" column

In [ ]:
credits_column_renamed = credits.rename(index=str, columns={'movie_id':"id"})
movies_df_merge = movies_df.merge(credits_column_renamed, on='id')
movies_df_merge.head()

There are a few columns which produce no value to our analysis. Hence we remove those columns

In [ ]:
movies_cleaned_df = movies_df_merge.drop(columns=['homepage','title_x', 'title_y', 'status', 'production_countries'])
movies_cleaned_df.head()

Let's check if our cleaned data has any missing values or not

In [ ]:
movies_cleaned_df.info()

## ***Using Weighted average for each movie's average rating***



<img src = "https://miro.medium.com/max/736/1*fGziZl2Do-VyQXSCPq_Y2Q.png"> </img>

Calculate all components based on above formula

In [ ]:
v = movies_cleaned_df['vote_count']
R = movies_cleaned_df['vote_average']
C = movies_cleaned_df['vote_average'].mean()
# Only consider movies which have 70 percentile or above vote counts
m = movies_cleaned_df['vote_count'].quantile(0.70)

In [ ]:
movies_cleaned_df['weighted_average'] = ((R*v)+(C*m))/(v+m)

In [ ]:
movies_cleaned_df.head()


In [ ]:
movies_cleaned_df.shape

In [ ]:
movies_cleaned_df.info()

In [ ]:
movie_sorted_ranking = movies_cleaned_df.sort_values('weighted_average', ascending=False)
movie_sorted_ranking[['original_title','vote_count', 'vote_average', 'weighted_average', 'popularity']]

It means that if you have seen the shawshank redemption, the next BESTSELLING movie which is recommended for you irrespective of any categorical filtering is The Godfather, Fight Club and so on.

But this mathematical model only considers viewers ratings.

To build a better recommendation system, we should also take the popularity into account

## Recommendation based on scaled weighted average and popularity score (Priority is given 50% to both)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaling = MinMaxScaler()
movie_scaled_df = scaling.fit_transform(movies_cleaned_df[['weighted_average', 'popularity']])
movie_normalized_df = pd.DataFrame(movie_scaled_df, columns=['weighted_average', 'popularity'])
movie_normalized_df.head()

In [ ]:
movies_cleaned_df[['normalized_weighted_average','normalized_popularity']] = movie_normalized_df
movies_cleaned_df.head()

In [ ]:
movies_cleaned_df['score'] = movies_cleaned_df['normalized_weighted_average']*0.5 + movies_cleaned_df['normalized_popularity']*0.5
movies_scored_df = movies_cleaned_df.sort_values(['score'], ascending=False)
movies_scored_df[['original_title', 'normalized_weighted_average', 'normalized_popularity', 'score']].head()

## ***Results of feature engineering***

We have created a recommendation engine which works for home page for a movie streaming web app

In [ ]:
movies_scored_df[['original_title', 'normalized_weighted_average', 'normalized_popularity', 'score']].head(20)